In [1]:
# packages
from google.cloud import bigquery
import os
import pickle
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# GDELT (Global Database of Events, Language, and Tone)

The GDELT [project](https://www.gdeltproject.org/) was first introduced by [Leetaru and
Schrodt (2013)](http://data.gdeltproject.org/documentation/ISA.2013.GDELT.pdf) based on earlier explorations (see for example [Schrodt (2010)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1643761)). 

It monitors print, broadcast, and web news media in over 100 languages from nearly all countries in the world. Its 2.0 Event Database contains a quarter billion geo-referenced event records in over 300 categories covering the entire world, updating every 15 minutes since February 2015. 
Before February 2015, the 1.0 Event Database releases daily updates. Historical data was collected back to 1979. 

The GDELT project pre-processes each of its sources and reports details on
the actors evolved, the type of the event, the number of mentions, sentiment score with respect to the source
and detailed information on the event’s location. Hence, answers to the questions: 

**What happened? Who was involved? Where did it happened? How is this talked about?**

While the GDELT project’s data have been widely used in social science research, it is important to be aware of
what it is and, accordingly, what it is not. The project captures an extensive picture of what is reported which
might diverge from what truly happened. This means that the data might include both false-positive as well as
numerous mentions of the same incident. In a reduced version, the project limits its data to one record of each
event type between actors per day. Additionally, the data are usually weighted by the total number of event
records collected per day since a higher number of events today than 20 years ago might just be a reflection
of technical advances. 

Compared with other event data, [Kwak and An (2016)](https://www.aaai.org/ocs/index.php/ICWSM/ICWSM16/paper/download/13014/12811) found that the GDELT project
seems over-state the number of events but also to have a superior method for the resolution of the geographic
locations of events which makes it very attractive. 

To classify actors and events, the GDELT project uses CAMEO’s coding scheme ([Schrodt, 2012](http://data.gdeltproject.org/documentation/CAMEO.Manual.1.1b3.pdf)).

In [6]:
# Set project_id to your Google Cloud Platform project ID.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service_account.json'

client = bigquery.Client()

project_id = "thesisau"

sql = """
SELECT  GLOBALEVENTID as id,
        SQLDATE as date, 
        Year as year, 
        Actor1Name as actor1_name,
        Actor1CountryCode as actor1_country, 
        Actor1KnownGroupCode as actor1_group,
        Actor1Religion1Code as actor1_religion1,
        Actor1Religion2Code as actor1_religion2,
        Actor1EthnicCode as actor1_ethnic,
        Actor1Type1Code as actor_type1,
        Actor1Type2Code as actor1_type2, 
        Actor1Type3Code as actor1_type3,
        Actor2Name as actor2_name, 
        Actor2CountryCode as actor2_country, 
        Actor2KnownGroupCode as actor2_group,
        Actor2Religion1Code as actor2_religion1,
        Actor2Religion2Code as actor2_religion2,
        Actor2EthnicCode as actor2_ethnic,
        Actor2Type1Code as actor2_type1, 
        Actor2Type2Code as actor2_type2, 
        Actor2Type3Code as actor2_type3, 
        EventCode as event_code,
        EventBaseCode as event_basecode,
        EventRootCode as event_rootcode,
        QuadClass as quad_class, 
        GoldsteinScale as goldstein_scale, 
        NumMentions as num_mentions, 
        NumSources as num_sources, 
        NumArticles as num_articles, 
        AvgTone as avg_tone,
        Actor1Geo_Type as actor1geo_res, 
        Actor1Geo_Fullname as actor1geo_name, 
        Actor1Geo_CountryCode as actor1geo_country, 
        Actor1Geo_Lat as actor1geo_lat,
        Actor1Geo_Long as actor1geo_long,
        Actor1Geo_FeatureID as actor1geo_id,
        Actor2Geo_Type as actor2geo_res, 
        Actor2Geo_Fullname as actor2geo_name, 
        Actor2Geo_CountryCode as actor2geo_country, 
        Actor2Geo_Lat as actor2geo_lat,
        Actor2Geo_Long as actor2geo_long,
        Actor2Geo_FeatureID as actor2geo_id,
        ActionGeo_Type as eventgeo_res, 
        ActionGeo_Fullname as eventgeo_name, 
        ActionGeo_CountryCode as eventgeo_country, 
        ActionGeo_Lat as eventgeo_lat,
        ActionGeo_Long as eventgeo_long,
        ActionGeo_FeatureID as eventgeo_id,
        DATEADDED as date_added, 
        SOURCEURL as source
FROM `gdelt-bq.full.events` 
WHERE ActionGeo_Type = 4 and ActionGeo_ADM1Code = 'GM07'
"""
# EventRootCode='14' and ActionGeo_CountryCode='TS' and Actor1CountryCode='TUN' and Actor1Geo_CountryCode = 'TS'

In [7]:
gdelt = client.query(sql, project=project_id).to_dataframe()

In [8]:
gdelt.head()

,id,date,year,actor1_name,actor1_country,actor1_group,actor1_religion1,actor1_religion2,actor1_ethnic,actor_type1,...,actor2geo_long,actor2geo_id,eventgeo_res,eventgeo_name,eventgeo_country,eventgeo_lat,eventgeo_long,eventgeo_id,date_added,source
0,935963142,20200719,2020,COMMANDER,None,None,None,None,None,MIL,...,NaN,None,4,"Totenkopf, Nordrhein-Westfalen, Germany",GM,51.45,8.78333,18650,20200719.0,https://www.edp24.co.uk/news/defenders-of-fran...
1,935980198,20200719,2020,SYDNEY,AUS,None,None,None,None,None,...,NaN,None,4,"Ennepe, Nordrhein-Westfalen, Germany",GM,51.20,7.46667,-1766618,20200719.0,https://www.dailymail.co.uk/news/article-85384...
2,935980200,20200719,2020,SYDNEY,AUS,None,None,None,None,None,...,NaN,None,4,"Ennepe, Nordrhein-Westfalen, Germany",GM,51.20,7.46667,-1766618,20200719.0,https://www.dailymail.co.uk/news/article-85384...
3,935980332,20200719,2020,GERMANY,DEU,None,None,None,None,None,...,NaN,None,4,"Ennepe, Nordrhein-Westfalen, Germany",GM,51.20,7.46667,-1766618,20200719.0,https://www.dailymail.co.uk/news/article-85384...
4,935980333,20200719,2020,GERMANY,DEU,None,None,None,None,None,...,NaN,None,4,"Mohne, Nordrhein-Westfalen, Germany",GM,51.45,7.95000,-1826677,20200719.0,https://www.dailymail.co.uk/news/article-85384...


In [12]:
len(gdelt)

247848

In [14]:
gdelt.year.min()

1920